In [21]:
import scanpy as sc
data_path = '../dataset/Chromium_FFPE_Human_Breast_Cancer_Chromium_FFPE_Human_Breast_Cancer_count_sample_filtered_feature_bc_matrix.h5'
sc_data = sc.read_10x_h5(data_path)
sc_data.X = sc_data.X.toarray()

/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [22]:
st_data_path = '../dataset/Xenium_breast_cancer_sample1_replicate1.h5ad'
st_data = sc.read_h5ad(st_data_path)
st_data

AnnData object with n_obs × n_vars = 167780 × 313
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'BT_FT_predictions_3407', 'BT_FT_predictions_42', 'BT_FT_predictions_476', 'BT_FT_predictions_761', 'BT_FT_predictions_9824', 'BT_FT_target_names_3407', 'BT_FT_target_names_42', 'BT_FT_target_names_476', 'BT_FT_target_names_761', 'BT_FT_target_names_9824', 'BT_FT_train_loss_3407', 'BT_FT_train_loss_42', 'BT_FT_train_loss_476', 'BT_FT_train_loss_761', 'BT_FT_train_loss_9824', 'BT_FT_val_loss_3407', 'BT_FT_val_loss_42', 'BT_FT_val_loss_476', 'BT_FT_val_loss_761', 'BT_FT_val_loss_9824', 'BT_FT_y_test_3407', 'BT_FT_y_test_42', 'BT_FT_y_test_476', 'BT_FT_y_test_761', 'BT_FT_y_test_9824', 'BT_ZS_predictions_3407', 'BT_ZS_predictions_42', 'BT_ZS_predictions_476', 'BT_ZS_predictions_761', 'BT_ZS_predictions_9824', '

In [23]:
# 检查 sc_data.var_names 是否有重复项
duplicates = sc_data.var_names[sc_data.var_names.duplicated()]
print(f"重复的基因: {duplicates}")
# 删除 sc_data.var_names 中的重复基因
sc_data = sc_data[:, ~sc_data.var_names.duplicated()]

重复的基因: Index(['TBCE', 'HSPA14', 'TMSB15B'], dtype='object')


In [24]:
st_data.X = st_data.X.toarray()

In [25]:
from turtle import st
import numpy as np
shared_genes = np.intersect1d(st_data.var_names, sc_data.var_names)
st_data = st_data[:, shared_genes]
sc.pp.filter_cells(st_data, min_counts=1)
sc.pp.filter_cells(sc_data, min_counts=1)
sc_data = sc_data.copy()
st_data = st_data.copy()

/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number
/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:168: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs["n_counts"] = number


In [26]:

st_data.obs['domain_id'] = 0
st_data.obs['domain_id'] = st_data.obs['domain_id'].astype('category')
st_data.obs['source'] = 'st'

sc_data.obs['source'] = 'sc'
sc_data.obs['domain_id'] = 1
sc_data.obs['domain_id'] = sc_data.obs['domain_id'].astype('category')


In [27]:
adata_cm = st_data.concatenate(sc_data, join='inner', batch_key='domain_id')

/tmp/ipykernel_2612008/3176745252.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata_cm = st_data.concatenate(sc_data, join='inner', batch_key='domain_id')


In [28]:
import anndata
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scvi.external import GIMVI
GIMVI.setup_anndata(st_data, batch_key='source')
GIMVI.setup_anndata(sc_data)

In [29]:
import uniport as up
sc.pp.normalize_total(adata_cm)
sc.pp.log1p(adata_cm)
up.batch_scale(adata_cm)
print(adata_cm)

AnnData object with n_obs × n_vars = 196719 × 307
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_counts', 'domain_id', 'source'
    var: 'gene_ids', 'feature_types-0', 'genome-0', 'feature_types-1', 'genome-1'
    uns: 'log1p'


In [30]:
sc.pp.normalize_total(st_data)
sc.pp.log1p(st_data)
up.batch_scale(st_data)
print(st_data)
sc.pp.normalize_total(sc_data)
sc.pp.log1p(sc_data)
up.batch_scale(sc_data)
print(sc_data)

AnnData object with n_obs × n_vars = 166354 × 307
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_counts', 'domain_id', 'source', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'BT_FT_predictions_3407', 'BT_FT_predictions_42', 'BT_FT_predictions_476', 'BT_FT_predictions_761', 'BT_FT_predictions_9824', 'BT_FT_target_names_3407', 'BT_FT_target_names_42', 'BT_FT_target_names_476', 'BT_FT_target_names_761', 'BT_FT_target_names_9824', 'BT_FT_train_loss_3407', 'BT_FT_train_loss_42', 'BT_FT_train_loss_476', 'BT_FT_train_loss_761', 'BT_FT_train_loss_9824', 'BT_FT_val_loss_3407', 'BT_FT_val_loss_42', 'BT_FT_val_loss_476', 'BT_FT_val_loss_761', 'BT_FT_val_loss_9824', 'BT_FT_y_test_3407', 'BT_FT_y_test_42', 'BT_FT_y_test_476', 'BT_FT_y_test_761', 'BT_FT_y_test_9824', 'BT_ZS_predictions_3407', 'BT_ZS_predictions_42', 'BT_ZS_pred

In [31]:
adatas = [st_data, sc_data]
adata = up.Run(adatas=adatas, adata_cm=adata_cm, model_info=True)

Device: cuda
Dataset 0: st
AnnData object with n_obs × n_vars = 166354 × 307
    obs: 'cell_type', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_counts', 'domain_id', 'source', '_scvi_batch', '_scvi_labels'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'BT_FT_predictions_3407', 'BT_FT_predictions_42', 'BT_FT_predictions_476', 'BT_FT_predictions_761', 'BT_FT_predictions_9824', 'BT_FT_target_names_3407', 'BT_FT_target_names_42', 'BT_FT_target_names_476', 'BT_FT_target_names_761', 'BT_FT_target_names_9824', 'BT_FT_train_loss_3407', 'BT_FT_train_loss_42', 'BT_FT_train_loss_476', 'BT_FT_train_loss_761', 'BT_FT_train_loss_9824', 'BT_FT_val_loss_3407', 'BT_FT_val_loss_42', 'BT_FT_val_loss_476', 'BT_FT_val_loss_761', 'BT_FT_val_loss_9824', 'BT_FT_y_test_3407', 'BT_FT_y_test_42', 'BT_FT_y_test_476', 'BT_FT_y_test_761', 'BT_FT_y_test_9824', 'BT_ZS_predictions_3407', 'BT_ZS_p

/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/uniport/function.py:443: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('Dataset {}:'.format(i), adata.obs[source_name][0])
2025-04-14 11:09:21,897 - root - INFO - model
VAE(
  (encoder): Encoder(
    (enc): ModuleList(
      (0): NN(
        (net): ModuleList(
          (0): Block(
            (fc): Linear(in_features=307, out_features=1024, bias=True)
            (norm): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act): ReLU()
          )
        )
      )
    )
    (mu_enc): ModuleList(
      (0): NN(
        (net): ModuleList(
          (0): Block(
            (fc): Linear(in_features=1024, out_features=16, bias=True)
          )
        )
      )
    )
    (var_enc): ModuleL

In [32]:
adata_predicted = up.Run(adata_cm=st_data, out='predict', pred_id=1)
adata_predicted.obsm['predict']


Device: cuda


/home/hanchuangyi/miniconda3/envs/scvi/lib/python3.10/site-packages/uniport/function.py:414: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  batches = adata_cm.obs[source_name][np.sort(idx)]


array([[0.04381977, 0.06471887, 0.04441128, ..., 0.1168635 , 0.28275442,
        0.20236127],
       [0.04854046, 0.07585114, 0.04788711, ..., 0.20931093, 0.41731685,
        0.34283775],
       [0.03250865, 0.04637736, 0.03878128, ..., 0.08811688, 0.20850605,
        0.13847712],
       ...,
       [0.08615073, 0.0953062 , 0.0682345 , ..., 0.20659529, 0.33975649,
        0.33758757],
       [0.09968153, 0.09920027, 0.06927311, ..., 0.19481735, 0.34004694,
        0.32216534],
       [0.08263823, 0.09529375, 0.06429652, ..., 0.23055452, 0.39195666,
        0.36559436]])

In [33]:
adata_predicted.obsm['predict'].shape

(166354, 18079)

In [34]:
import anndata as ad
adata_imputed = ad.AnnData(
    X=adata_predicted.obsm['predict'],
    obs=st_data.obs,
    uns=st_data.uns,
)

In [35]:
adata_imputed.var_names = sc_data.var_names
adata_imputed.var_names

Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1', 'HES4', 'ISG15',
       'AGRN', 'RNF223', 'C1orf159',
       ...
       'MT-ND2', 'MT-CO2', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4',
       'MT-ND5', 'MT-ND6', 'MT-CYB'],
      dtype='object', length=18079)

In [36]:
adata_imputed.X = adata_imputed.X.astype(np.float16)

In [37]:
adata_imputed.write_h5ad('../dataset/uniport_imputed_Xenium_breast_cancer_sample1_replicate1.h5ad')

... storing 'source' as categorical
